In [1]:
import networkx as nx, math, sys, numpy as np, matplotlib.pyplot as plt, time, csv, os
from tqdm import tqdm
from collections import defaultdict

In [2]:
phi = 0.75
gRed = 0.15
gBlue = 0.15

In [3]:
def read_community(filename):
    with open(filename, "r") as file:
        lines = file.readlines()

    # Extract the number of communities
    num_communities = int(lines[0].strip())

    # Process the data for each node
    for line in lines[1:]:
        node, community = map(int, line.strip().split())
        if community == 1:
            red.append(node)
        elif community == 0:
            blue.append(node)

In [4]:
def makeGraph(filename):
    with open(filename, "r") as file:
        num_nodes = int(file.readline().strip())  # Extract the number of nodes

        for line in file:
            line = line.strip()
            line = line.split("\t")
            source = int(line[0])
            target = int(line[1])
            G.add_edge(source, target)
        for node in G.nodes:
            if G.has_edge(node, node):
                G.remove_edge(node, node)
    #             print(node)

In [5]:
def outRB():
    for node in G.nodes:
        G.nodes[node]['outR'] = 0
        G.nodes[node]['outB'] = 0
        G.nodes[node]['outRList'] = []
        G.nodes[node]['outBList'] = []
        for n in G.neighbors(node):
            if n in red:
                G.nodes[node]['outR'] += 1
                G.nodes[node]['outRList'].append(n)
            elif n in blue:
                G.nodes[node]['outB'] += 1
                G.nodes[node]['outBList'].append(n)

In [6]:
def initlR():
    tmplR = []
    for node in G.nodes:
        if G.out_degree(node) != 0 and G.nodes[node]['outR'] / G.out_degree(node) < phi and G.nodes[node]['outR'] != 0:
            tmplR.append(node)
    return tmplR

In [7]:
def updateD(node):
    for i in G.nodes[node]['outRList']:
#         D[node][i] = (phi - (G.nodes[node]['outR'] / G.degree(node))) / G.nodes[node]['outR']
        D[node][i] = (phi/G.nodes[node]['outR']) - (1/G.out_degree(node))
    for i in G.nodes[node]['outBList']:
#         D[node][i] = ((G.nodes[node]['outR'] / G.degree(node)) - phi) / G.nodes[node]['outB']
        D[node][i] = ((1-phi)/G.nodes[node]['outB']) - (1/G.out_degree(node))

In [8]:
def cmpFormula():
    tmpGains = {}
    pRlist = np.inner(Q, color)
    
    for node in lR:
        g = gamma[node][node]
        sPkR = 0
        sPkB = 0
        sPkxR = 0
        sPkxB = 0
        fTerm = g/((1-g)*(phi - (G.nodes[node]['outR'] / G.out_degree(node))))
    
        dR = 1/G.nodes[node]['outR']
        dB = 1/G.nodes[node]['outB']

        for n in G.nodes[node]['outRList']:
            sPkR += pRlist[n]
            sPkxR += Q[n][node]
        for n in G.nodes[node]['outBList']:
            sPkB += pRlist[n]
            sPkxB += Q[n][node]

        numerator = dR*sPkR - dB*sPkB
        denominator = fTerm-(dR*sPkxR - dB*sPkxB)
        tmpGains[node] = pr[node]*numerator/denominator
    return tmpGains

In [9]:
graphFile = '/out_graph.txt'
G = nx.DiGraph()
red = []
blue = []
read_community(sys.path[0] + '/out_community.txt')
nodesList = sorted(red+blue)
G.add_nodes_from(nodesList)
makeGraph(sys.path[0] + graphFile)
# G = nx.convert_node_labels_to_integers(G)

outRB()
lR = initlR()



P = nx.adjacency_matrix(G)
P = P.toarray()
row_sums = P.sum(axis=1)
P = (P / row_sums[:, np.newaxis])

where_are_NaNs = np.isnan(P)
P[where_are_NaNs] = 0

u = np.empty(len(P)) 
u.fill(1/len(P))
u = np.transpose(u)

# Handle sink nodes
d = np.zeros(len(P))

for node in G.nodes:
    if G.out_degree(node) == 0:
        d[node] = 1
P = P + np.outer(d, u)

I = np.identity(len(P))
D = np.zeros(shape=(len(P), len(P)))

# Random jump vector
v = np.empty(len(P)) 
v.fill(1/len(G.nodes))

# Vector with 1 over red nodes
color = np.zeros(len(P))
for node in red:
    color[node] = 1
    
gamma = np.zeros(len(P)) 
for node in G.nodes:
    if node in red:
        gamma[node] = gRed
    else:
        gamma[node] = gBlue
gamma = np.diag(gamma)

# intrinsic opinions
s = np.zeros(len(P))
for node in G.nodes:
    if node in red:
        s[node] = 1
    else:
        s[node] = -1

/var/folders/qf/yh559xpx4dv12z0mkyw5cvr80000gn/T/ipykernel_5217/1544359872.py:19: RuntimeWarning: invalid value encountered in divide
  P = (P / row_sums[:, np.newaxis])


In [10]:
Q = (np.linalg.inv(I-((I-gamma).dot(P)))).dot(gamma)
z = Q @ s

In [11]:
start_time = time.time()
results = defaultdict(lambda: {"Fairness": 0.0, "CostV": 0.0, "CostS": 0.0})

fair = np.zeros(len(P)) 

fairNodes = []
for fNodes in (range(0, len(lR) + 1)):
    print(fNodes, "fair nodes are: ", fairNodes[:fNodes])
    Q = (np.linalg.inv(I-((I-gamma).dot(P+D)))).dot(gamma)
    zPrime = Q @ s
    c = (zPrime - z) ** 2
    results[fNodes]["CostV"] = np.sum(c)/len(G.nodes)
    # results[fNodes]["CostS"] = np.sum(c*fair)/fNodes
    results[fNodes]["CostS"] = np.sum(c*fair) / max(1, fNodes)
    pr = v.dot(Q)
    prRed = np.inner(pr, color)
    print("PageRank allocated to red group: ", prRed)
    if len(lR) > 0:
        gains = cmpFormula()
        nFair = max(gains, key=gains.get)
        print(nFair, gains[nFair], "-----")
        lR.remove(nFair)
        fairNodes.append(nFair)
        updateD(nFair)
        fair[nFair] = 1
    results[fNodes]["Fairness"] = prRed
print("--- %s seconds ---" % (time.time() - start_time))


data = results
keys = list(data.keys())
filtered_keys = [keys[0]] + [key for i, key in enumerate(keys[1:], start=1) if data[key]['Fairness'] >= data[keys[i-1]]['Fairness']]

results = {key: data[key] for key in filtered_keys}

0 fair nodes are:  []
PageRank allocated to red group:  0.3337480440108953
565 0.00564812506717287 -----
1 fair nodes are:  [565]
PageRank allocated to red group:  0.33939616907806813
767 0.004659335195493922 -----
2 fair nodes are:  [565, 767]
PageRank allocated to red group:  0.344055504273562
1005 0.0042486515250502975 -----
3 fair nodes are:  [565, 767, 1005]
PageRank allocated to red group:  0.34830415579861235
575 0.003956431267083747 -----
4 fair nodes are:  [565, 767, 1005, 575]
PageRank allocated to red group:  0.3522605870656961
837 0.00359226622954849 -----
5 fair nodes are:  [565, 767, 1005, 575, 837]
PageRank allocated to red group:  0.3558528532952445
618 0.003324471212638627 -----
6 fair nodes are:  [565, 767, 1005, 575, 837, 618]
PageRank allocated to red group:  0.3591773245078832
847 0.0029379221961394436 -----
7 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847]
PageRank allocated to red group:  0.3621152467040226
889 0.0028966241261141417 -----
8 fair nodes are: 

PageRank allocated to red group:  0.40758851778577365
512 0.0006258479150429548 -----
40 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512]
PageRank allocated to red group:  0.4082143657008166
638 0.0006125028794957925 -----
41 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638]
PageRank allocated to red group:  0.4088268685803125
457 0.0005873934785334419 -----
42 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457]
PageRank allocated to red group:  0.409

PageRank allocated to red group:  0.41746474255995647
477 0.0002788119678858419 -----
64 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477]
PageRank allocated to red group:  0.4177435545278423
605 0.0002608081578671567 -----
65 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605]
PageRank allocated to red group:  0.4180043626857095
259 0.0002532811700446954 -----
66 fair nodes are:  [565, 767, 1005, 57

PageRank allocated to red group:  0.4218664646684048
593 0.00021851249415161435 -----
82 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593]
PageRank allocated to red group:  0.42208497716255633
403 0.0002175986625658047 -----
83 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552

PageRank allocated to red group:  0.42498300076563883
608 0.00016924797630063295 -----
98 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593, 403, 594, 750, 149, 111, 342, 582, 646, 436, 287, 197, 612, 407, 100, 231, 608]
PageRank allocated to red group:  0.42515224874193963
397 0.00016911876219361446 -----
99 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 5

PageRank allocated to red group:  0.4271245010525704
532 0.00013501695791041468 -----
112 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593, 403, 594, 750, 149, 111, 342, 582, 646, 436, 287, 197, 612, 407, 100, 231, 608, 397, 207, 468, 696, 668, 471, 312, 657, 335, 303, 272, 606, 390, 532]
PageRank allocated to red group:  0.4272595180104808
267 0.00013155239395837116 -----
113 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 4

PageRank allocated to red group:  0.4286018449760346
504 0.00010830050641512016 -----
124 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593, 403, 594, 750, 149, 111, 342, 582, 646, 436, 287, 197, 612, 407, 100, 231, 608, 397, 207, 468, 696, 668, 471, 312, 657, 335, 303, 272, 606, 390, 532, 267, 49, 551, 317, 66, 301, 375, 473, 502, 138, 757, 504]
PageRank allocated to red group:  0.4287101454824497
106 0.00010554299550681431 -----
125 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742,

PageRank allocated to red group:  0.42958016267416
295 4.2897697086974345e-05 -----
136 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593, 403, 594, 750, 149, 111, 342, 582, 646, 436, 287, 197, 612, 407, 100, 231, 608, 397, 207, 468, 696, 668, 471, 312, 657, 335, 303, 272, 606, 390, 532, 267, 49, 551, 317, 66, 301, 375, 473, 502, 138, 757, 504, 106, 62, 395, 642, 426, 372, 481, 515, 417, 562, 29, 295]
PageRank allocated to red group:  0.42962306037124703
16 4.118931525293533e-05 -----
137 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 8

PageRank allocated to red group:  0.42998420377667346
327 2.2393461552203867e-05 -----
148 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889, 496, 1127, 1078, 945, 1058, 1029, 671, 1151, 796, 790, 994, 829, 974, 841, 1159, 571, 693, 995, 1074, 940, 497, 742, 519, 881, 524, 1009, 932, 564, 956, 591, 535, 512, 638, 457, 672, 838, 917, 868, 839, 771, 421, 919, 610, 554, 494, 380, 654, 588, 747, 705, 567, 244, 458, 466, 214, 477, 605, 259, 389, 883, 601, 404, 371, 552, 630, 490, 346, 765, 597, 408, 836, 691, 713, 593, 403, 594, 750, 149, 111, 342, 582, 646, 436, 287, 197, 612, 407, 100, 231, 608, 397, 207, 468, 696, 668, 471, 312, 657, 335, 303, 272, 606, 390, 532, 267, 49, 551, 317, 66, 301, 375, 473, 502, 138, 757, 504, 106, 62, 395, 642, 426, 372, 481, 515, 417, 562, 29, 295, 16, 116, 277, 109, 247, 476, 28, 269, 170, 67, 511, 327]
PageRank allocated to red group:  0.43000659723822565
328 2.0008157668079532e-05 -----
149 fair nodes are:  [565, 767, 1005, 575, 837, 618, 847, 889,